In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
import re
import math
from scipy.stats import t
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from pylab import *
import statsmodels.api as sm

print("Random Forest on Airline On-Time Arrivals")
homepath="./data/"
csv=pd.read_csv(homepath+"flight.csv",index_col=26) 
data=pd.DataFrame(csv)
#Loading raw csv file from 
#http://transtats.bts.gov/Tables.asp?DB_ID=120&DB_Name=Airline%20On-Time%20Performance%20Data&DB_Short_Name=On-Time

result_col="ARR_DEL15" #Define the input column
dropRow=["DEP_DEL15","ARR_DELAY","YEAR","MONTH","CANCELLED","CARRIER_DELAY","WEATHER_DELAY",
         "NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"] #Drop columns which do not need for this analysis

# Filter rows
data=data[data["CANCELLED"]==0] #Filter rows which "CANCELLED" is 0
data.fillna(0, inplace=True) #Fill all Nan cells with 0

result=np.asarray(data[result_col],dtype=float)
data.drop(result_col,axis=1,inplace=True)
for i in range(len(dropRow)):
    data.drop(dropRow[i],axis=1,inplace=True)

#Split data into train and test set - Training set will be used for training the predictor while the test set is for validating the result
x_train,x_test,y_train,y_test=train_test_split(data,result,test_size=0.3,random_state=1) 
clf=RandomForestClassifier(n_estimators=121).fit(x_train,y_train) #We ploted the OOB graph and found 121 estimators give the lowest error
Test_scores=clf.score(x_test,y_test) #Returning the score is the mean accuracy
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, clf.predict_proba(x_test)[:,1]) #Ploting a ROC Curve
auc = roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) #Returning the Area of ROC

print ("Test_scores",Test_scores)
print ("AUC",auc)
print ("false_positive_rate",false_positive_rate)
print ("true_positive_rate",true_positive_rate)
print ("thresholds",thresholds)

#Save false_positive_rate, true_positive_rate, thresholds in a csv
fpr_tpr=np.c_[false_positive_rate,true_positive_rate,thresholds]
fpr_tpr=pd.DataFrame(fpr_tpr,columns=["false_positive_rate","true_positive_rate","thresholds"])

#Calculate the importances of each variable and save in a csv
importances=clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0)

datacolumns=list(data.columns)
importancedata=np.c_[datacolumns,importances]
importance_chart=pd.DataFrame(importancedata)

#Save the Test score in the same csv
importance_chart.loc[len(importance_chart)]=["Test_scores",Test_scores]
importance_chart.loc[len(importance_chart)]=["AUC",auc]


#I am also interested in what factor "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY" 
#have highest correlation to the Arrival Delay

def de_mean(x):
    xmean = mean(x)
    return [xi - xmean for xi in x]


def covariance(x, y):
    n = len(x)
    return dot(de_mean(x), de_mean(y)) / (n-1)

def correlation(x,y):
    stddevx = x.std()
    stddevy = y.std()
    return covariance(x,y)/stddevx /stddevy 

#================================================================================================
print("Correlation")
#I am also interested in what factor "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY" 
#have the highest correlation to the Arrival Delay
data=pd.DataFrame(csv)
data=data[data["CANCELLED"]==0] #Filter rows which "CANCELLED" is 0
data.fillna(0, inplace=True) #Fill all Nan cells with 0
data=data[data["ARR_DEL15"]==1] #Filter the rows which are delayed for more than 15 minutes
result=data["ARR_DELAY"] 
cor_factor=["CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
cor_matrix=pd.DataFrame([],columns=["Factor","Correlation","Mean","Stddev"])

for i in range(len(cor_factor)):
    cor_matrix=cor_matrix.append({"Factor":cor_factor[i],
                                  "Correlation":correlation(result,data[cor_factor[i]]),
                                  "Mean":data[cor_factor[i]].mean(),
                                  "Stddev":data[cor_factor[i]].std()},ignore_index=True)

importance_chart.to_csv(homepath+"importances.csv")
cor_matrix.to_csv(homepath+"correlation.csv")
fpr_tpr.to_csv(homepath+"/fpr_tpr.csv")


#================================================================================================
print("Multivariance")
#I am interested in how each factor affect the Arrival Delay 
#have the highest correlation to the Arrival Delay
data=pd.DataFrame(csv)
data=data[data["CANCELLED"]==0] #Filter rows which "CANCELLED" is 0
data.fillna(0, inplace=True) #Fill all Nan cells with 0
result=data["ARR_DELAY"]
cor_factor=["CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]

X=data[["CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]]
Y=data[["ARR_DELAY"]]
X1 = sm.add_constant(X)
est = sm.OLS(Y, X1).fit()

print (est.summary())
#================================================================================================
